Loading the Data

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Load the data
df_0 = pd.read_csv(r"C:\Users\pingk\Downloads\fadhli nitip\asik_NIR_DIST_3b.csv")

In [3]:
df_0.head()

,prov_char,country_char,wavenumber,11544,11540,11536,11532,11528,11524,11520,...,3964,3960,3956,3952,tgp_name,dgp_name,fgp_name,country_name,thnoth_name,thnoth_name_encoded
0,KAX,ID,ID-KAX-088-2401-001,0.290252,0.290122,0.290044,0.290029,0.290042,0.289978,0.289851,...,3.558086,3.542757,3.530849,3.501222,Group 3,Group 2,Group 4,Indonesia,Non-Thai,0
1,KBX,ID,ID-KBX-068-2306-003,0.522845,0.522798,0.522826,0.522669,0.522414,0.522217,0.522070,...,3.680422,3.655416,3.629388,3.606223,Group 3,Group 2,Group 4,Indonesia,Non-Thai,0
2,BBX,ID,ID-BBX-095-2401-004,-0.144108,-0.144008,-0.143893,-0.143777,-0.143686,-0.143667,-0.143678,...,3.650478,3.623180,3.614590,3.581875,Group 3,Group 2,Group 3,Indonesia,Non-Thai,0
3,SUX,ID,ID-SUX-074-2311-001,0.333905,0.333777,0.333785,0.333700,0.333522,0.333453,0.333446,...,3.601331,3.566453,3.539593,3.512285,Group 3,Group 2,Group 3,Indonesia,Non-Thai,0
4,SUX,ID,ID-SUX-073-2311-001,1.522433,1.521460,1.521133,1.521072,1.521494,1.522300,1.521840,...,4.103365,4.125110,4.113971,4.055192,Group 3,Group 2,Group 3,Indonesia,Non-Thai,0


Selecting Regions of Interest

In [4]:
# Define the regions of interest
regions_of_interest = [
    (4641, 4681), (4867, 5510), (5657, 5826), (7057, 7097),
    (7169, 7209), (8238, 8278)
]

# Extract columns corresponding to the regions of interest
columns_to_focus = []
for start, end in regions_of_interest:
    columns_to_focus.extend([col for col in df_0.columns[4:-7] if start <= float(col) <= end])

# Create a new DataFrame with the selected regions
df_0_selected_regions = df_0[columns_to_focus]

# Combine the selected regions with the target column and other relevant columns
df_0_selected_regions = pd.concat([df_0[['thnoth_name', 'prov_char']], df_0_selected_regions], axis=1)

# Save the DataFrame for further processing
df_0_selected_regions.to_csv('data/data file 2b/data_1.csv', index=False)


In [5]:
df_0_selected_regions.head()

,thnoth_name,prov_char,4680,4676,4672,4668,4664,4660,4656,4652,...,8276,8272,8268,8264,8260,8256,8252,8248,8244,8240
0,Non-Thai,KAX,1.145221,1.161998,1.183733,1.206279,1.221476,1.219369,1.195433,1.155057,...,0.693094,0.696532,0.699208,0.701041,0.702028,0.702149,0.701310,0.699461,0.696617,0.692720
1,Non-Thai,KBX,1.100158,1.117663,1.140348,1.163641,1.178847,1.175626,1.149462,1.106238,...,0.827193,0.830521,0.833110,0.834886,0.835800,0.835858,0.835008,0.833129,0.830211,0.826278
2,Non-Thai,BBX,0.873460,0.891096,0.914072,0.937998,0.954190,0.951890,0.926066,0.882441,...,0.370219,0.373736,0.376443,0.378311,0.379332,0.379466,0.378665,0.376875,0.374042,0.370139
3,Non-Thai,SUX,1.014767,1.031323,1.052854,1.075045,1.089551,1.086283,1.060706,1.018413,...,0.694977,0.698438,0.701122,0.702996,0.703988,0.704090,0.703278,0.701477,0.698633,0.694729
4,Non-Thai,SUX,1.520511,1.536409,1.557498,1.579159,1.592749,1.588327,1.561828,1.519022,...,1.606612,1.609559,1.611674,1.613343,1.614402,1.614270,1.612957,1.610735,1.607516,1.603303


FUNCTIONS

Baseline Correction

In [6]:
from scipy.signal import savgol_filter

# Function for baseline correction with dynamic window length
def baseline_correction(spectrum, default_window_length=15, polyorder=3):
    spectrum_length = len(spectrum)
    if spectrum_length < default_window_length:
        window_length = spectrum_length // 2 * 2 + 1  # Make window length odd and less than the size of the spectrum
    else:
        window_length = default_window_length
    baseline = savgol_filter(spectrum, window_length, polyorder, mode='nearest')
    corrected_spectrum = spectrum - baseline
    return corrected_spectrum

# Apply baseline correction
df_baseline_corrected_v0 = df_0_selected_regions.copy()
for col in columns_to_focus:
    df_baseline_corrected_v0[col] = baseline_correction(df_baseline_corrected_v0[col])

# Save the baseline corrected data
df_baseline_corrected_v0.to_csv('data/data file 2b/data_1_bslcrct.csv', index=False)

SavGol Smoothing

In [8]:
# Function for Savitzky-Golay smoothing
def savitzky_golay_smoothing(spectrum, default_window_length=11, polyorder=2):
    window_length = min(default_window_length, len(spectrum) // 2 * 2 + 1)  # Make window length odd and less than or equal to the size of the spectrum
    if window_length < 3:  # Ensure window length is at least 3
        window_length = 3
    return savgol_filter(spectrum, window_length, polyorder, mode='nearest')  # Set mode to 'nearest'

# Apply smoothing
df_smoothed_v0 = df_baseline_corrected_v0.copy()
for col in columns_to_focus:
    df_smoothed_v0[col] = savitzky_golay_smoothing(df_smoothed_v0[col])

# Save the smoothed data
df_smoothed_v0.to_csv('data/data file 2b/data_1_smoothed.csv', index=False)


Normalization

In [9]:
# Function for normalization (Min-Max scaling)
def min_max_normalization(spectrum):
    return (spectrum - np.min(spectrum)) / (np.max(spectrum) - np.min(spectrum))

# Apply normalization
df_normalized_v0 = df_smoothed_v0.copy()
for col in columns_to_focus:
    df_normalized_v0[col] = min_max_normalization(df_normalized_v0[col])

# Save the normalized data
df_normalized_v0.to_csv('data/data file 2b/data_1_normalized.csv', index=False)


Derivatization (np.gradient)

In [10]:
# Calculate the first derivative using np.gradient
data_spectrum = df_normalized_v0.iloc[:, 2:].values
first_derivative_np = np.gradient(data_spectrum, axis=1)

# Calculate the second derivative using np.gradient
second_derivative_np = np.gradient(first_derivative_np, axis=1)

# Convert the results back to DataFrame
data_1_der_np = pd.DataFrame(first_derivative_np, columns=df_normalized_v0.columns[2:])
data_2_der_np = pd.DataFrame(second_derivative_np, columns=df_normalized_v0.columns[2:])

# Combine the first two columns from the original dataset with the np.gradient derivatives

# Extract the first two columns
first_two_columns = df_normalized_v0.iloc[:, :2]

# Combine the first two columns with the derivatives
data_1_der_combined = pd.concat([first_two_columns, data_1_der_np], axis=1)
data_2_der_combined = pd.concat([first_two_columns, data_2_der_np], axis=1)

# Export the combined data to CSV
data_1_der_combined.to_csv('data/data file 2b/data_1_1_der.csv', index=False)
data_2_der_combined.to_csv('data/data file 2b/data_1_2_der.csv', index=False)

Derivatization (SavGol)

In [11]:
# Extract the spectrum data
data_spectrum = df_normalized_v0.iloc[:, 2:].values

# Apply Savitzky-Golay filter for the first derivative
first_derivative_savgol = savgol_filter(data_spectrum, window_length=5, polyorder=2, deriv=1, axis=1)

# Apply Savitzky-Golay filter for the second derivative
second_derivative_savgol = savgol_filter(data_spectrum, window_length=5, polyorder=2, deriv=2, axis=1)

# Convert the results back to DataFrame
data_1_der_savgol = pd.DataFrame(first_derivative_savgol, columns=df_normalized_v0.columns[2:])
data_2_der_savgol = pd.DataFrame(second_derivative_savgol, columns=df_normalized_v0.columns[2:])

# Extract the first two columns
first_two_columns = df_normalized_v0.iloc[:, :2]

# Combine the first two columns with the Savitzky-Golay derivatives
data_1_der_savgol_combined = pd.concat([first_two_columns, data_1_der_savgol], axis=1)
data_2_der_savgol_combined = pd.concat([first_two_columns, data_2_der_savgol], axis=1)

# Export the combined data to CSV
data_1_der_savgol_combined.to_csv('data/data file 2b/data_1_1_der_savgol.csv', index=False)
data_2_der_savgol_combined.to_csv('data/data file 2b/data_1_2_der_savgol.csv', index=False)

SNV

In [12]:
def snv(spectrum):
    return (spectrum - np.mean(spectrum)) / np.std(spectrum)

# Apply SNV to the selected regions
df_snv = df_0_selected_regions.copy()
for col in columns_to_focus:
    df_snv[col] = snv(df_snv[col])

# Save the SNV data
df_snv.to_csv('data/data file 2b/data_1_snv.csv', index=False)


Random Normal Variate (RNV)

In [13]:
def rnv(spectrum):
    random_noise = np.random.normal(0, np.std(spectrum), spectrum.shape)
    return spectrum + random_noise

# Apply RNV to the selected regions
df_rnv = df_0_selected_regions.copy()
for col in columns_to_focus:
    df_rnv[col] = rnv(df_rnv[col])

# Save the RNV data
df_rnv.to_csv('data/data file 2b/data_1_rnv.csv', index=False)


Multiplicative Scatter Correction (MSC)

In [14]:
# Select only the spectral df_0_selected_regions columns (excluding 'thnoth_name', 'prov_char', and the last two columns)
spectral_df_0_selected_regions_columns = df_0_selected_regions.columns[2:]
spectral_df_0_selected_regions = df_0_selected_regions[spectral_df_0_selected_regions_columns]

# Calculate the mean spectrum across all samples
mean_spectrum = spectral_df_0_selected_regions.mean(axis=0)

# Perform Mean Centering (MSC)
msc_spectral_df_0_selected_regions = spectral_df_0_selected_regions - mean_spectrum

# Add back the non-spectral columns to the df_0_selected_regionsframe
msc_df_0_selected_regions = df_0_selected_regions[['thnoth_name', 'prov_char']].copy()
msc_df_0_selected_regions = pd.concat([msc_df_0_selected_regions, msc_spectral_df_0_selected_regions], axis=1)

# Save the MSC preprocessed df_0_selected_regions to a new CSV file
msc_df_0_selected_regions.to_csv('data/data file 2b/data_1_msc.csv', index=False)

# Display the first few rows of the MSC preprocessed df_0_selected_regions
#print(msc_df_0_selected_regions.head())

Classification and Evaluation (40-fold)

In [17]:
# Define the target variable
target = 'thnoth_name'

# Ensure columns_to_focus are correctly identified
numeric_cols_df_0 = df_0_selected_regions.select_dtypes(include=[np.number]).columns.tolist()
columns_to_focus = numeric_cols_df_0  # Ensure columns are correctly selected

# Classification and evaluation function using 40-fold CV
def classify_and_evaluate(df, columns):
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import cross_val_score
    from sklearn.preprocessing import LabelEncoder

    # Encode target variable
    le = LabelEncoder()
    y = le.fit_transform(df[target])

    # Define features
    X = df[columns]

    # Initialize the classifier
    model = RandomForestClassifier(n_estimators=100, random_state=42)

    # Perform cross-validation
    scores = cross_val_score(model, X, y, cv=40)

    # Print the results
    print(f'Cross-Validation Accuracy: {np.mean(scores)}')


In [19]:

# Evaluate each preprocessing method
print("Evaluation for No Preprocessing:")
classify_and_evaluate(df_0_selected_regions, columns_to_focus)

Evaluation for No Preprocessing:
Cross-Validation Accuracy: 0.6871829710144928


In [21]:
print("Evaluation for Baseline Correction:")
df_baseline_corrected_v01 = pd.read_csv('data/data file 2b/data_1_bslcrct.csv')
classify_and_evaluate(df_baseline_corrected_v01, columns_to_focus)

Evaluation for Baseline Correction:
Cross-Validation Accuracy: 0.5912590579710144


In [22]:
print("Evaluation for Smoothing:")
df_smoothed_v01 = pd.read_csv('data/data file 2b/data_1_smoothed.csv')
classify_and_evaluate(df_smoothed_v01, columns_to_focus)

Evaluation for Smoothing:
Cross-Validation Accuracy: 0.5933876811594203


In [23]:
print("Evaluation for Normalization:")
df_normalized_v01 = pd.read_csv('data/data file 2b/data_1_normalized.csv')
classify_and_evaluate(df_normalized_v01, columns_to_focus)

Evaluation for Normalization:
Cross-Validation Accuracy: 0.5933876811594203


In [25]:
print("Evaluation for 1-Derivative Spectroscopy:")
data_1_der_combined_v01 = pd.read_csv('data/data file 2b/data_1_1_der.csv')
classify_and_evaluate(data_1_der_combined_v01, columns_to_focus)

Evaluation for 1-Derivative Spectroscopy:
Cross-Validation Accuracy: 0.8411684782608695


In [26]:
print("Evaluation for 2-Derivative Spectroscopy:")
data_2_der_combined_v01 = pd.read_csv('data/data file 2b/data_1_2_der.csv')
classify_and_evaluate(data_2_der_combined_v01, columns_to_focus)

Evaluation for 2-Derivative Spectroscopy:
Cross-Validation Accuracy: 0.8201539855072465


In [27]:
print("Evaluation for 1-SG-Derivative Spectroscopy:")
data_1_der_savgol_combined_v01 = pd.read_csv('data/data file 2b/data_1_1_der_savgol.csv')
classify_and_evaluate(data_1_der_savgol_combined_v01, columns_to_focus)

Evaluation for 1-SG-Derivative Spectroscopy:
Cross-Validation Accuracy: 0.8316576086956522


In [28]:
print("Evaluation for 2-SG-Derivative Spectroscopy:")
data_2_der_savgol_combined_v01 = pd.read_csv('data/data file 2b/data_1_2_der_savgol.csv')
classify_and_evaluate(data_2_der_savgol_combined_v01, columns_to_focus)

Evaluation for 2-SG-Derivative Spectroscopy:
Cross-Validation Accuracy: 0.8287137681159422


In [29]:
print("Evaluation for SNV:")
df_snv_v01 = pd.read_csv('data/data file 2b/data_1_snv.csv')
classify_and_evaluate(df_snv_v01, columns_to_focus)

print("Evaluation for RNV:")
df_rnv_v01 = pd.read_csv('data/data file 2b/data_1_rnv.csv')
classify_and_evaluate(df_rnv_v01, columns_to_focus)

print("Evaluation for MSC:")
df_msc_v01 = pd.read_csv('data/data file 2b/data_1_msc.csv')
classify_and_evaluate(df_msc_v01, columns_to_focus)

Evaluation for SNV:
Cross-Validation Accuracy: 0.6871829710144928
Evaluation for RNV:
Cross-Validation Accuracy: 0.5071557971014492
Evaluation for MSC:
Cross-Validation Accuracy: 0.6871829710144928


Classification and Evaluation (40-fold) -with extra detailed

In [30]:
# Define the target variable
target = 'thnoth_name'

# Ensure columns_to_focus are correctly identified
numeric_cols_df_0 = df_0_selected_regions.select_dtypes(include=[np.number]).columns.tolist()
columns_to_focus = numeric_cols_df_0  # Ensure columns are correctly selected

# Classification and evaluation function using 40-fold CV with detailed metrics
def classify_and_evaluate(df, columns):
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import cross_val_score, StratifiedKFold
    from sklearn.preprocessing import LabelEncoder
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
    import numpy as np

    # Encode target variable
    le = LabelEncoder()
    y = le.fit_transform(df[target])

    # Define features
    X = df[columns]

    # Initialize the classifier
    model = RandomForestClassifier(n_estimators=100, random_state=42)

    # Initialize Stratified K-Fold Cross-Validation
    skf = StratifiedKFold(n_splits=40)

    # Arrays to store results
    accuracies = []
    precisions = []
    recalls = []
    f1_scores = []
    conf_matrices = []

    # Perform cross-validation
    for train_idx, test_idx in skf.split(X, y):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        # Train the model
        model.fit(X_train, y_train)

        # Predict and evaluate
        y_pred = model.predict(X_test)
        accuracies.append(accuracy_score(y_test, y_pred))
        precisions.append(precision_score(y_test, y_pred, average=None))
        recalls.append(recall_score(y_test, y_pred, average=None))
        f1_scores.append(f1_score(y_test, y_pred, average=None))
        conf_matrices.append(confusion_matrix(y_test, y_pred))

    # Calculate mean scores
    mean_accuracy = np.mean(accuracies)
    mean_precision = np.mean(precisions, axis=0)
    mean_recall = np.mean(recalls, axis=0)
    mean_f1 = np.mean(f1_scores, axis=0)
    mean_conf_matrix = np.mean(conf_matrices, axis=0)

    # Print the results
    print(f'Cross-Validation Accuracy: {mean_accuracy}')
    print(f'Precision per class: {mean_precision}')
    print(f'Recall per class: {mean_recall}')
    print(f'F1-score per class: {mean_f1}')


In [31]:
# Evaluate each preprocessing method
print("Evaluation for No Preprocessing:")
classify_and_evaluate(df_0_selected_regions, columns_to_focus)

print("Evaluation for Baseline Correction:")
df_baseline_corrected_v02 = pd.read_csv('data/data file 2b/data_1_bslcrct.csv')
classify_and_evaluate(df_baseline_corrected_v02, columns_to_focus)

print("Evaluation for Smoothing:")
df_smoothed_v02 = pd.read_csv('data/data file 2b/data_1_smoothed.csv')
classify_and_evaluate(df_smoothed_v02, columns_to_focus)

print("Evaluation for Normalization:")
df_normalized_v02 = pd.read_csv('data/data file 2b/data_1_normalized.csv')
classify_and_evaluate(df_normalized_v02, columns_to_focus)

print("Evaluation for 1-Derivative Spectroscopy:")
df_derivative_v02 = pd.read_csv('data/data file 2b/data_1_1_der.csv')
classify_and_evaluate(df_derivative_v02, columns_to_focus)

print("Evaluation for 2-Derivative Spectroscopy:")
data_2_der_combined_v02 = pd.read_csv('data/data file 2b/data_1_2_der.csv')
classify_and_evaluate(data_2_der_combined_v02, columns_to_focus)

print("Evaluation for 1-SG-Derivative Spectroscopy:")
data_1_der_savgol_combined_v02 = pd.read_csv('data/data file 2b/data_1_1_der_savgol.csv')
classify_and_evaluate(data_1_der_savgol_combined_v02, columns_to_focus)

print("Evaluation for 2-SG-Derivative Spectroscopy:")
data_2_der_savgol_combined_v02 = pd.read_csv('data/data file 2b/data_1_2_der_savgol.csv')
classify_and_evaluate(data_2_der_savgol_combined_v02, columns_to_focus)

Evaluation for No Preprocessing:
Cross-Validation Accuracy: 0.6871829710144928
Precision per class: [0.7089437  0.67677392]
Recall per class: [0.64772727 0.72613636]
F1-score per class: [0.67206709 0.69703752]
Evaluation for Baseline Correction:
Cross-Validation Accuracy: 0.5912590579710144
Precision per class: [0.59410964 0.59282087]
Recall per class: [0.54109848 0.6405303 ]
F1-score per class: [0.55693867 0.60898542]
Evaluation for Smoothing:
Cross-Validation Accuracy: 0.5933876811594203
Precision per class: [0.59585311 0.59481052]
Recall per class: [0.54242424 0.6407197 ]
F1-score per class: [0.56007546 0.61027071]
Evaluation for Normalization:
Cross-Validation Accuracy: 0.5933876811594203
Precision per class: [0.59585311 0.59481052]
Recall per class: [0.54242424 0.6407197 ]
F1-score per class: [0.56007546 0.61027071]
Evaluation for 1-Derivative Spectroscopy:
Cross-Validation Accuracy: 0.8411684782608695
Precision per class: [0.85779591 0.85701772]
Recall per class: [0.83712121 0.84

In [32]:
print("Evaluation for SNV:")
df_snv_v02 = pd.read_csv('data/data file 2b/data_1_snv.csv')
classify_and_evaluate(df_snv_v02, columns_to_focus)

print("Evaluation for RNV:")
df_rnv_v02 = pd.read_csv('data/data file 2b/data_1_rnv.csv')
classify_and_evaluate(df_rnv_v02, columns_to_focus)

print("Evaluation for MSC:")
df_msc_v02 = pd.read_csv('data/data file 2b/data_1_msc.csv')
classify_and_evaluate(df_msc_v02, columns_to_focus)

Evaluation for SNV:
Cross-Validation Accuracy: 0.6871829710144928
Precision per class: [0.7089437  0.67677392]
Recall per class: [0.64772727 0.72613636]
F1-score per class: [0.67206709 0.69703752]
Evaluation for RNV:
Cross-Validation Accuracy: 0.5071557971014492
Precision per class: [0.51623482 0.49980187]
Recall per class: [0.46060606 0.55113636]
F1-score per class: [0.47596215 0.51654288]
Evaluation for MSC:
Cross-Validation Accuracy: 0.6871829710144928
Precision per class: [0.7089437  0.67677392]
Recall per class: [0.64772727 0.72613636]
F1-score per class: [0.67206709 0.69703752]


Classification and Evaluation using LOGO-CV

In [33]:
# Classification and evaluation function using LOGO-CV with detailed metrics
def classify_and_evaluate_logo_cv_detailed(df, columns):
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import LeaveOneGroupOut
    from sklearn.preprocessing import LabelEncoder
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
    
    # Encode target variable
    le = LabelEncoder()
    y = le.fit_transform(df[target])
    
    # Define features
    X = df[columns]
    
    # Initialize the classifier
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    
    # Initialize LOGO-CV
    logo = LeaveOneGroupOut()
    groups = df['prov_char']
    
    # Arrays to store results
    accuracies = []
    precisions = []
    recalls = []
    f1_scores = []
    all_y_test = []
    all_y_pred = []
    
    # Perform LOGO-CV
    for train_idx, test_idx in logo.split(X, y, groups=groups):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        
        # Train the model
        model.fit(X_train, y_train)
        
        # Predict and evaluate
        y_pred = model.predict(X_test)
        accuracies.append(accuracy_score(y_test, y_pred))
        all_y_test.extend(y_test)
        all_y_pred.extend(y_pred)
        
    # Calculate overall metrics
    mean_accuracy = np.mean(accuracies)
    precision = precision_score(all_y_test, all_y_pred, average=None)
    recall = recall_score(all_y_test, all_y_pred, average=None)
    f1 = f1_score(all_y_test, all_y_pred, average=None)
    conf_matrix = confusion_matrix(all_y_test, all_y_pred)
    
    # Print the results
    print(f'Mean Accuracy: {mean_accuracy}')
    print(f'Precision per class: {precision}')
    print(f'Recall per class: {recall}')
    print(f'F1-score per class: {f1}')
    
    # Return confusion matrix for presentation
    return conf_matrix

In [34]:
# Evaluate each preprocessing method using LOGO-CV with detailed metrics
print("LOGO-CV Evaluation for No Preprocessing:")
conf_matrix_no_preprocessing_v03 = classify_and_evaluate_logo_cv_detailed(df_0_selected_regions, columns_to_focus)

print("LOGO-CV Evaluation for Baseline Correction:")
df_baseline_corrected_v03 = pd.read_csv('data/data file 2b/data_1_bslcrct.csv')
conf_matrix_baseline_v03 = classify_and_evaluate_logo_cv_detailed(df_baseline_corrected_v03, columns_to_focus)

print("LOGO-CV Evaluation for Smoothing:")
df_smoothed_v03 = pd.read_csv('data/data file 2b/data_1_smoothed.csv')
conf_matrix_smoothing_v03 = classify_and_evaluate_logo_cv_detailed(df_smoothed_v03, columns_to_focus)

print("LOGO-CV Evaluation for Normalization:")
df_normalized_v03 = pd.read_csv('data/data file 2b/data_1_normalized.csv')
conf_matrix_normalization_v03 = classify_and_evaluate_logo_cv_detailed(df_normalized_v03, columns_to_focus)

print("LOGO-CV Evaluation for 1-Derivative Spectroscopy:")
df_1_derivative_v03 = pd.read_csv('data/data file 2b/data_1_1_der.csv')
conf_matrix_1_derivative_v03 = classify_and_evaluate_logo_cv_detailed(df_1_derivative_v03, columns_to_focus)

print("LOGO-CV Evaluation for 2-Derivative Spectroscopy:")
df_2_derivative_v03 = pd.read_csv('data/data file 2b/data_1_2_der.csv')
conf_matrix_2_derivative_v03 = classify_and_evaluate_logo_cv_detailed(df_2_derivative_v03, columns_to_focus)

print("LOGO-CV Evaluation for 1-SG-Derivative Spectroscopy:")
df_1_der_savgol_combined_v03 = pd.read_csv('data/data file 2b/data_1_1_der_savgol.csv')
conf_matrix_1_sg_v03 = classify_and_evaluate_logo_cv_detailed(df_1_der_savgol_combined_v03, columns_to_focus)

print("LOGO-CV Evaluation for 2-SG-Derivative Spectroscopy:")
df_2_der_savgol_combined_v03 = pd.read_csv('data/data file 2b/data_1_2_der_savgol.csv')
conf_matrix_2_sg_v03 = classify_and_evaluate_logo_cv_detailed(df_2_der_savgol_combined_v03, columns_to_focus)

print("LOGO-CV Evaluation for SNV:")
df_snv_v03 = pd.read_csv('data/data file 2b/data_1_snv.csv')
conf_matrix_snv_v03 = classify_and_evaluate_logo_cv_detailed(df_snv_v03, columns_to_focus)

print("LOGO-CV Evaluation for RNV:")
df_rnv_v03 = pd.read_csv('data/data file 2b/data_1_rnv.csv')
conf_matrix_rnv_v03 = classify_and_evaluate_logo_cv_detailed(df_rnv_v03, columns_to_focus)

print("LOGO-CV Evaluation for MSC:")
df_msc_v03 = pd.read_csv('data/data file 2b/data_1_msc.csv')
conf_matrix_msc_v03 = classify_and_evaluate_logo_cv_detailed(df_msc_v03, columns_to_focus)

LOGO-CV Evaluation for No Preprocessing:
Mean Accuracy: 0.5801241907237283
Precision per class: [0.49579832 0.4974271 ]
Recall per class: [0.37659574 0.61702128]
F1-score per class: [0.4280532  0.55080722]
LOGO-CV Evaluation for Baseline Correction:
Mean Accuracy: 0.5074279246074703
Precision per class: [0.39304813 0.42932862]
Recall per class: [0.31276596 0.51702128]
F1-score per class: [0.34834123 0.46911197]
LOGO-CV Evaluation for Smoothing:
Mean Accuracy: 0.575664986849123
Precision per class: [0.47869674 0.48428835]
Recall per class: [0.40638298 0.55744681]
F1-score per class: [0.43958573 0.51829871]
LOGO-CV Evaluation for Normalization:
Mean Accuracy: 0.5753659437869222
Precision per class: [0.47738693 0.48339483]
Recall per class: [0.40425532 0.55744681]
F1-score per class: [0.43778802 0.51778656]
LOGO-CV Evaluation for 1-Derivative Spectroscopy:
Mean Accuracy: 0.8545845339928044
Precision per class: [0.80472103 0.79957806]
Recall per class: [0.79787234 0.80638298]
F1-score per 

Display Confusion Matrices

In [35]:
# Function to display confusion matrix in a tabular format
def display_confusion_matrix(conf_matrix, class_labels):
    df_cm = pd.DataFrame(conf_matrix, index=class_labels, columns=class_labels)
    print(df_cm)

# Ensure LabelEncoder is defined and class labels are set
from sklearn.preprocessing import LabelEncoder

# Define the target variable and fit LabelEncoder
target = 'thnoth_name'
le = LabelEncoder()
le.fit(df_0_selected_regions[target])
class_labels = le.classes_

In [36]:
# Display confusion matrices for each preprocessing method
print("Confusion Matrix for No Preprocessing:")
display_confusion_matrix(conf_matrix_no_preprocessing_v03, class_labels)

print("Confusion Matrix for Baseline Correction:")
display_confusion_matrix(conf_matrix_baseline_v03, class_labels)

print("Confusion Matrix for Smoothing:")
display_confusion_matrix(conf_matrix_smoothing_v03, class_labels)

print("Confusion Matrix for Normalization:")
display_confusion_matrix(conf_matrix_normalization_v03, class_labels)

print("Confusion Matrix for Derivative Spectroscopy:")
display_confusion_matrix(conf_matrix_1_derivative_v03, class_labels)

print("Confusion Matrix for Derivative Spectroscopy:")
display_confusion_matrix(conf_matrix_2_derivative_v03, class_labels)

print("Confusion Matrix for Derivative Spectroscopy:")
display_confusion_matrix(conf_matrix_1_sg_v03, class_labels)

print("Confusion Matrix for Derivative Spectroscopy:")
display_confusion_matrix(conf_matrix_2_sg_v03, class_labels)

print("Confusion Matrix for SNV:")
display_confusion_matrix(conf_matrix_snv_v03, class_labels)

print("Confusion Matrix for RNV:")
display_confusion_matrix(conf_matrix_rnv_v03, class_labels)

print("Confusion Matrix for MSC:")
display_confusion_matrix(conf_matrix_msc_v03, class_labels)

Confusion Matrix for No Preprocessing:
          Non-Thai  Thai
Non-Thai       177   293
Thai           180   290
Confusion Matrix for Baseline Correction:
          Non-Thai  Thai
Non-Thai       147   323
Thai           227   243
Confusion Matrix for Smoothing:
          Non-Thai  Thai
Non-Thai       191   279
Thai           208   262
Confusion Matrix for Normalization:
          Non-Thai  Thai
Non-Thai       190   280
Thai           208   262
Confusion Matrix for Derivative Spectroscopy:
          Non-Thai  Thai
Non-Thai       375    95
Thai            91   379
Confusion Matrix for Derivative Spectroscopy:
          Non-Thai  Thai
Non-Thai       361   109
Thai            94   376
Confusion Matrix for Derivative Spectroscopy:
          Non-Thai  Thai
Non-Thai       365   105
Thai            94   376
Confusion Matrix for Derivative Spectroscopy:
          Non-Thai  Thai
Non-Thai       363   107
Thai            93   377
Confusion Matrix for SNV:
          Non-Thai  Thai
Non-Thai       1